### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('./Datasets/movies_clean.csv')
reviews = pd.read_csv('./Datasets/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items[:500000].groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [3]:
# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [24]:
n_users = ratings_mat.shape[0] # number of rows in the matrix
n_movies = ratings_mat.shape[1] # number of movies in the matrix
num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
num_ratings

51

In [18]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iter in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2 * diff * movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2 * diff * user_mat[i, k])
                        
        # print results
        print("%d \t\t %f" % (iter+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [19]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 46.065934
2 		 20.998331
3 		 6.587952
4 		 3.288672
5 		 2.859008
6 		 2.746249
7 		 2.666986
8 		 2.591267
9 		 2.510915
10 		 2.422542


In [20]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 8.3896177   7.40257395  8.67644017  8.10154334]
 [ 9.94076492  8.55136306 10.07240042  9.6921973 ]
 [ 6.62399029  6.14232357  6.57355956  6.75595977]
 [ 7.37908489  6.48078413  6.65842195  7.64021293]
 [ 9.6604949   8.63853219 10.13902617  9.30982162]
 [ 8.91345604  7.43008323  8.83491663  8.67219459]
 [ 9.41047628  7.91257909  9.77926207  8.98555369]
 [ 6.93755873  5.72609528  7.68754027  6.32534936]
 [ 9.34395807  8.26341158  9.78904417  9.0312115 ]
 [ 9.56933711  8.68144876  9.52824859  9.65339061]
 [ 8.52400995  6.80112304  8.7603072   7.90716606]
 [ 9.95679703  8.46921525 10.12820589  9.61113344]
 [ 7.75427495  6.69501557  8.20175007  7.42194053]]
[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [21]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.909607
2 		 23.312796
3 		 7.776147
4 		 3.585199
5 		 2.985302
6 		 2.888301
7 		 2.846145
8 		 2.810724
9 		 2.774593
10 		 2.735572
11 		 2.692583
12 		 2.644817
13 		 2.591524
14 		 2.531965
15 		 2.465423
16 		 2.391237
17 		 2.308852
18 		 2.217882
19 		 2.118185
20 		 2.009939
21 		 1.893716
22 		 1.770541
23 		 1.641912
24 		 1.509783
25 		 1.376470
26 		 1.244517
27 		 1.116500
28 		 0.994816
29 		 0.881484
30 		 0.777995
31 		 0.685229
32 		 0.603465
33 		 0.532447
34 		 0.471506
35 		 0.419701
36 		 0.375947
37 		 0.339130
38 		 0.308180
39 		 0.282125
40 		 0.260112
41 		 0.241412
42 		 0.225416
43 		 0.211620
44 		 0.199613
45 		 0.189059
46 		 0.179688
47 		 0.171280
48 		 0.163658
49 		 0.156678
50 		 0.150225
51 		 0.144205
52 		 0.138544
53 		 0.133180
54 		 0.128066
55 		 0.123162
56 		 0.118439
57 		 0.113873
58 		 0.109444
59 		 0.105138
60 		 0.100946
61 		 0.096858
62 		 0.092870
63 		 0.088978
64 		

In [22]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.00300238  7.00149402 10.00039155  7.99679731]
 [ 9.99811377  7.99903637  9.99972254 10.00198537]
 [ 5.00056148  8.00027727  5.00006852  7.99939319]
 [ 9.99973792  7.99987023  0.99996683 10.00028103]
 [ 9.00010705 10.00004478 10.00000344  8.99987661]
 [10.00064519  4.00031928  9.00008137  9.99930828]
 [ 9.0001006   8.00003717  9.99999696  8.99987902]
 [ 5.99876288  3.99937504  9.99982379  6.00130259]
 [ 9.99999407  8.9999947   9.99999662  8.00000466]
 [10.00102877 10.00050909  9.00013029  8.99889962]
 [ 7.99874444  6.99936535  9.99982275  7.00132604]
 [ 8.9985731   8.99929237  9.99981201 10.00151243]
 [ 9.00147756  5.00074716  9.00020817  6.99843736]]
[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [23]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan,  7., 10.,  8.],
        [10.,  8., 10., 10.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8.,  1., 10.],
        [ 9., 10., 10.,  9.],
        [10.,  4.,  9., 10.],
        [ 9.,  8., 10.,  9.],
        [ 6.,  4., 10.,  6.],
        [10.,  9., 10.,  8.],
        [10., 10.,  9.,  9.],
        [ 8.,  7., 10.,  7.],
        [ 9.,  9., 10., 10.],
        [ 9.,  5.,  9.,  7.]])

In [25]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 50.377976
2 		 26.260319
3 		 9.272165
4 		 3.924055
5 		 3.014031
6 		 2.856958
7 		 2.795682
8 		 2.746618
9 		 2.696255
10 		 2.640653
11 		 2.577867
12 		 2.506623
13 		 2.426010
14 		 2.335439
15 		 2.234702
16 		 2.124065
17 		 2.004357
18 		 1.877036
19 		 1.744184
20 		 1.608427
21 		 1.472744
22 		 1.340215
23 		 1.213708
24 		 1.095600
25 		 0.987559
26 		 0.890453
27 		 0.804386
28 		 0.728832
29 		 0.662832
30 		 0.605206
31 		 0.554721
32 		 0.510228
33 		 0.470727
34 		 0.435398
35 		 0.403594
36 		 0.374820
37 		 0.348701
38 		 0.324948
39 		 0.303335
40 		 0.283672
41 		 0.265797
42 		 0.249559
43 		 0.234817
44 		 0.221434
45 		 0.209279
46 		 0.198225
47 		 0.188152
48 		 0.178947
49 		 0.170505
50 		 0.162732
51 		 0.155541
52 		 0.148856
53 		 0.142609
54 		 0.136743
55 		 0.131207
56 		 0.125958
57 		 0.120960
58 		 0.116183
59 		 0.111602
60 		 0.107196
61 		 0.102948
62 		 0.098844
63 		 0.094874
64 		

In [26]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 7.687292780128397:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [27]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 19.632723
2 		 9.068013
3 		 6.341771
4 		 4.959357
5 		 4.102623
6 		 3.513284
7 		 3.079753
8 		 2.745339
9 		 2.478037
10 		 2.258509
11 		 2.074487
12 		 1.917843
13 		 1.782967
14 		 1.665820
15 		 1.563386
16 		 1.473332
17 		 1.393801
18 		 1.323278
19 		 1.260502
20 		 1.204412


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [29]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14771.

The number of ratings made for user-movie pairs that didn't have ratings is 27902229


In [30]:
# Test your results against the solution
assert num_ratings == 14771, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 27902229, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
